In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)
engine

Engine(sqlite:///:memory:)

In [2]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

In [3]:
print(Base.metadata)
Base.registry

MetaData()


### Mapped classes

In [4]:
from sqlalchemy.orm import Mapped, mapped_column, relationship
from sqlalchemy import String, ForeignKey
from typing import Optional, List


class Cliente(Base):
    __tablename__ = 'CLIENTE'

    id: Mapped[int] = mapped_column(primary_key=True)
    nome: Mapped[str] = mapped_column(String(50))
    sobrenome: Mapped[Optional[str]]

    contas: Mapped[List['Conta']] = relationship(back_populates='cliente')

    def __repr__(self) -> str:
        return f'Cliente(id={self.id!r}, nome={self.nome!r}, sobrenome={self.sobrenome!r}, contas={self.contas!r})'


class Conta(Base):
    __tablename__ = 'CONTA'

    id: Mapped[int] = mapped_column(primary_key=True)
    saldo: Mapped[float] = mapped_column(default=0)
    id_cliente = mapped_column(ForeignKey('CLIENTE.id'))

    cliente: Mapped[Cliente] = relationship(back_populates='contas')

    def __repr__(self) -> str:
        return f'Conta(id={self.id!r}, saldo={self.saldo!r}, id_cliente={self.id_cliente!r}, cliente={self.cliente.nome!r})'

In [5]:
Base.metadata.create_all(engine)

2023-11-30 20:08:45,657 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,658 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CLIENTE")
2023-11-30 20:08:45,658 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:08:45,659 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CLIENTE")
2023-11-30 20:08:45,660 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:08:45,660 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CONTA")
2023-11-30 20:08:45,661 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:08:45,662 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CONTA")
2023-11-30 20:08:45,662 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:08:45,663 INFO sqlalchemy.engine.Engine 
CREATE TABLE "CLIENTE" (
	id INTEGER NOT NULL, 
	nome VARCHAR(50) NOT NULL, 
	sobrenome VARCHAR, 
	PRIMARY KEY (id)
)


2023-11-30 20:08:45,664 INFO sqlalchemy.engine.Engine [no key 0.00073s] ()
2023-11-30 20:08:45,665 INFO sqlalchemy.engine.Engine 
CREATE T

In [6]:
from sqlalchemy import inspect

inspect(engine).get_table_names()

2023-11-30 20:08:45,673 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,674 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-11-30 20:08:45,674 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:08:45,674 INFO sqlalchemy.engine.Engine ROLLBACK


['CLIENTE', 'CONTA']

In [7]:
alberto = Cliente(nome='alberto', sobrenome='frigatto')
alberto

Cliente(id=None, nome='alberto', sobrenome='frigatto', contas=[])

In [8]:
gustavo = Cliente(nome='gustavo')
gustavo

Cliente(id=None, nome='gustavo', sobrenome=None, contas=[])

In [9]:
from sqlalchemy.orm import Session

session = Session(engine)
session

In [10]:
session.add(alberto)
session.add(gustavo)

In [11]:
session.new

IdentitySet([Cliente(id=None, nome='alberto', sobrenome='frigatto', contas=[]), Cliente(id=None, nome='gustavo', sobrenome=None, contas=[])])

In [12]:
session.flush()

2023-11-30 20:08:45,714 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,716 INFO sqlalchemy.engine.Engine INSERT INTO "CLIENTE" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-30 20:08:45,717 INFO sqlalchemy.engine.Engine [generated in 0.00011s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('alberto', 'frigatto')
2023-11-30 20:08:45,718 INFO sqlalchemy.engine.Engine INSERT INTO "CLIENTE" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-30 20:08:45,718 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('gustavo', None)


In [13]:
gustavo

2023-11-30 20:08:45,728 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-30 20:08:45,728 INFO sqlalchemy.engine.Engine [generated in 0.00075s] (2,)


Cliente(id=2, nome='gustavo', sobrenome=None, contas=[])

In [14]:
session.get(Cliente, 2)

Cliente(id=2, nome='gustavo', sobrenome=None, contas=[])

In [15]:
alberto.id

1

In [16]:
gustavo.id

2

In [17]:
session.commit()
session.close()

2023-11-30 20:08:45,762 INFO sqlalchemy.engine.Engine COMMIT


In [18]:
from sqlalchemy import select

print(select(Cliente))

SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE"


In [19]:
with Session(engine) as session:
    result = session.scalars(select(Cliente))

    for cliente in result:
        print(f'{cliente.id} -> {cliente.nome} -> {cliente.sobrenome}')

2023-11-30 20:08:45,778 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,779 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE"
2023-11-30 20:08:45,780 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ()
1 -> alberto -> frigatto
2 -> gustavo -> None
2023-11-30 20:08:45,781 INFO sqlalchemy.engine.Engine ROLLBACK


In [20]:
with Session(engine) as session:
    vinicius = Cliente(nome='Vinicius', sobrenome='Alcântara')
    # session.add(vinicius)

    print(vinicius in session)

False


In [21]:
with Session(engine) as session:
    print(session.execute(select(Cliente).filter_by(nome='gustavo')).scalar_one())
    print(session.scalar(select(Cliente).filter_by(nome='gustavo')))

2023-11-30 20:08:45,799 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,801 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE" 
WHERE "CLIENTE".nome = ?
2023-11-30 20:08:45,802 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ('gustavo',)
2023-11-30 20:08:45,804 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-30 20:08:45,805 INFO sqlalchemy.engine.Engine [cached since 0.07838s ago] (2,)
Cliente(id=2, nome='gustavo', sobrenome=None, contas=[])
2023-11-30 20:08:45,808 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE" 
WHERE "CLIENTE".nome = ?
2023-11-30 20:08:45,808 INFO sqlalchemy.engine.Engine [cached since 0.006314s ago] ('gustavo',)
2023-11-30 20:08:45,809 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "

In [22]:
session = Session(engine)

alberto = session.get(Cliente, 1)
alberto

2023-11-30 20:08:45,817 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,819 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-30 20:08:45,819 INFO sqlalchemy.engine.Engine [generated in 0.00069s] (1,)
2023-11-30 20:08:45,821 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-30 20:08:45,822 INFO sqlalchemy.engine.Engine [cached since 0.09446s ago] (1,)


Cliente(id=1, nome='alberto', sobrenome='frigatto', contas=[])

In [23]:
alberto.nome = 'Alberto Luíz'

In [24]:
session.dirty

IdentitySet([Cliente(id=1, nome='Alberto Luíz', sobrenome='frigatto', contas=[])])

In [25]:
alberto_sobrenome = session.scalar(select(Cliente.sobrenome).where(Cliente.id == 2))
alberto_sobrenome

2023-11-30 20:08:45,842 INFO sqlalchemy.engine.Engine UPDATE "CLIENTE" SET nome=? WHERE "CLIENTE".id = ?
2023-11-30 20:08:45,843 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ('Alberto Luíz', 1)
2023-11-30 20:08:45,844 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".sobrenome 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-30 20:08:45,844 INFO sqlalchemy.engine.Engine [generated in 0.00061s] (2,)


In [26]:
session.delete(alberto)

In [27]:
session.execute(select(Cliente).filter_by(nome='Alberto Luíz')).first()

2023-11-30 20:08:45,857 INFO sqlalchemy.engine.Engine DELETE FROM "CLIENTE" WHERE "CLIENTE".id = ?
2023-11-30 20:08:45,857 INFO sqlalchemy.engine.Engine [generated in 0.00065s] (1,)
2023-11-30 20:08:45,858 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE" 
WHERE "CLIENTE".nome = ?
2023-11-30 20:08:45,859 INFO sqlalchemy.engine.Engine [cached since 0.05719s ago] ('Alberto Luíz',)


In [28]:
fred = Cliente(nome='Fred', sobrenome='Boat')

session.add(fred)
session.commit()

2023-11-30 20:08:45,865 INFO sqlalchemy.engine.Engine INSERT INTO "CLIENTE" (nome, sobrenome) VALUES (?, ?)
2023-11-30 20:08:45,866 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ('Fred', 'Boat')
2023-11-30 20:08:45,867 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
fred

2023-11-30 20:08:45,873 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,874 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-30 20:08:45,875 INFO sqlalchemy.engine.Engine [generated in 0.00061s] (3,)
2023-11-30 20:08:45,876 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-30 20:08:45,877 INFO sqlalchemy.engine.Engine [cached since 0.1498s ago] (3,)


Cliente(id=3, nome='Fred', sobrenome='Boat', contas=[])

In [30]:
fred.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1c8fb4ed3d0>,
 'nome': 'Fred',
 'id': 3,
 'sobrenome': 'Boat',
 'contas': []}

In [31]:
session.rollback()

2023-11-30 20:08:45,891 INFO sqlalchemy.engine.Engine ROLLBACK


In [32]:
fred.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1c8fb4ed3d0>}

In [33]:
fred

2023-11-30 20:08:45,902 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,902 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-30 20:08:45,903 INFO sqlalchemy.engine.Engine [cached since 0.02851s ago] (3,)
2023-11-30 20:08:45,904 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-30 20:08:45,904 INFO sqlalchemy.engine.Engine [cached since 0.177s ago] (3,)


Cliente(id=3, nome='Fred', sobrenome='Boat', contas=[])

In [34]:
session.delete(fred)

In [35]:
session.rollback()

2023-11-30 20:08:45,917 INFO sqlalchemy.engine.Engine ROLLBACK


In [36]:
fred

2023-11-30 20:08:45,922 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,923 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-30 20:08:45,923 INFO sqlalchemy.engine.Engine [cached since 0.04949s ago] (3,)
2023-11-30 20:08:45,925 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-30 20:08:45,925 INFO sqlalchemy.engine.Engine [cached since 0.1984s ago] (3,)


Cliente(id=3, nome='Fred', sobrenome='Boat', contas=[])

In [37]:
fred in session

True

In [38]:
session.close()

2023-11-30 20:08:45,940 INFO sqlalchemy.engine.Engine ROLLBACK


In [39]:
with Session(engine) as session:
    result = session.scalars(select(Cliente))

    for cliente in result:
        print(f'{cliente.id} -> {cliente.nome} -> {cliente.sobrenome}')

2023-11-30 20:08:45,948 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,948 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE"
2023-11-30 20:08:45,949 INFO sqlalchemy.engine.Engine [cached since 0.17s ago] ()
2 -> gustavo -> None
3 -> Fred -> Boat
2023-11-30 20:08:45,950 INFO sqlalchemy.engine.Engine ROLLBACK


In [40]:
fred

Cliente(id=3, nome='Fred', sobrenome='Boat', contas=[])

In [41]:
conta_fred = Conta(saldo=0, cliente=fred)
conta_fred

Conta(id=None, saldo=0, id_cliente=None, cliente='Fred')

In [42]:
session = Session(engine)

session.add(conta_fred)
session.commit()

2023-11-30 20:08:45,970 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,971 INFO sqlalchemy.engine.Engine INSERT INTO "CONTA" (saldo, id_cliente) VALUES (?, ?)
2023-11-30 20:08:45,972 INFO sqlalchemy.engine.Engine [generated in 0.00063s] (0.0, 3)
2023-11-30 20:08:45,973 INFO sqlalchemy.engine.Engine COMMIT


In [43]:
conta_fred

2023-11-30 20:08:45,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:08:45,980 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE "CONTA".id = ?
2023-11-30 20:08:45,981 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (1,)
2023-11-30 20:08:45,982 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-30 20:08:45,982 INFO sqlalchemy.engine.Engine [cached since 0.1084s ago] (3,)


Conta(id=1, saldo=0.0, id_cliente=3, cliente='Fred')

In [44]:
fred.contas[0]

2023-11-30 20:08:45,990 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-30 20:08:45,991 INFO sqlalchemy.engine.Engine [cached since 0.2635s ago] (3,)


Conta(id=1, saldo=0.0, id_cliente=3, cliente='Fred')

In [45]:
session.close()

2023-11-30 20:08:45,999 INFO sqlalchemy.engine.Engine ROLLBACK
